# Análisis de Sentimientos en Twitter

---


# IMPORTANTE: CAMBIAR DE ENV A tsa
https://www.youtube.com/watch?v=9Sfs7Fbvtdk&t=220s


## Introducción:

Busqué "twitter sentiment analysis" en google y las páginas más relevantes fueron:

* [This Is How Twitter Sees The World : Sentiment Analysis Part One](https://towardsdatascience.com/the-real-world-as-seen-on-twitter-sentiment-analysis-part-one-5ac2d06b63fb): usa la base de datos [Sentiment140](http://help.sentiment140.com/) y Naive Bayes para clasificar en sentimiento positivo y negativo.

* [Comprehensive Hands on Guide to Twitter Sentiment Analysis with dataset and code](https://www.analyticsvidhya.com/blog/2018/07/hands-on-sentiment-analysis-dataset-python/): Usa BOW y TF-IDF para identificar tweets racistas o sexistas. Sacó el dataset de [acá](https://datahack.analyticsvidhya.com/contest/practice-problem-twitter-sentiment-analysis/).

* [Twitter Sentiment Analysis using Python](https://www.geeksforgeeks.org/twitter-sentiment-analysis-using-python/): este usa [TextBlob](https://textblob.readthedocs.io/en/dev/) y [tweepy](http://docs.tweepy.org/en/v3.5.0/) para descargar tweets y clasifica en positivos y negativos. [Este](https://www.freecodecamp.org/news/how-to-build-a-twitter-sentiments-analyzer-in-python-using-textblob-948e1e8aae14/) también hace lo mismo. [Este](https://www.youtube.com/watch?v=27P268Q7pE0) también. Todos hacen lo mismo igual.

* [Competencia de Kaggle](https://www.kaggle.com/c/twitter-sentiment-analysis2/overview) para clasificar texto en positivo, negativo y neutral. Ya viene el dataset ahí.

* [Sentiment Analysis of Twitter](https://monkeylearn.com/blog/sentiment-analysis-of-twitter/): Este tiene unformación bastante piola.

Artículos encontrados en google scholar:

* [Twitter Sentiment Analysis:The Good the Bad and the OMG!](http://localhost:8888/files/BecaNLP/twitter_sentiment_analysis/Twitter%20Sentiment%20Analysis%3AThe%20Good%20the%20Bad%20and%20the%20OMG!.pdf): Usa tres datasets distintos y hace extracción de features. Quiere ver cuán bien funciona Twitter para sentiment analysis.

* [Twitter Sentiment Classification using Distant Supervision](http://localhost:8888/files/BecaNLP/twitter_sentiment_analysis/Twitter%20Sentiment%20Classification%20using%20Distant%20Supervision.pdf): Este dice ser el primer paper de sentiment analysis en twitter.

* [Semantic Sentiment Analysis of Twitter](http://localhost:8888/files/BecaNLP/twitter_sentiment_analysis/Semantic%20Sentiment%20Analysis%20of%20Twitter.pdf)

* [A System for Real-time Twitter Sentiment Analysis of2012 U.S. Presidential Election Cycle](http://localhost:8888/files/BecaNLP/twitter_sentiment_analysis/A%20System%20for%20Real-time%20Twitter%20Sentiment%20Analysis%20of2012%20U.S.%20Presidential%20Election%20Cycle.pdf)

* [Twitter Sentiment Analysis with Deep Convolutional Neural Networks](http://localhost:8888/files/BecaNLP/twitter_sentiment_analysis/Twitter%20Sentiment%20Analysis%20with%20Deep%20Convolutional%20Neural%20Networks.pdf)

**IMPORTANTE: investigar un poco más el tema de NLTK: http://www.nltk.org/data.html**

## Twitter con TextBlob y Tweepy

In [9]:
import re 
import tweepy 
from tweepy import OAuthHandler 
from textblob import TextBlob 
# import nltk
# import ssl

# try:
#     _create_unverified_https_context = ssl._create_unverified_context
# except AttributeError:
#     pass
# else:
#     ssl._create_default_https_context = _create_unverified_https_context
    
# nltk.download('punkt')

# nltk.download('averaged_perceptron_tagger')

Ejemplo de uso de TextBlob:

In [8]:
example = TextBlob("Python is a high-level, general-purpose programming language.")

print(example.words)
print(example.sentences)
print(example.tags)

['Python', 'is', 'a', 'high-level', 'general-purpose', 'programming', 'language']
[Sentence("Python is a high-level, general-purpose programming language.")]
[('Python', 'NNP'), ('is', 'VBZ'), ('a', 'DT'), ('high-level', 'JJ'), ('general-purpose', 'JJ'), ('programming', 'NN'), ('language', 'NN')]


Ejemplo de uso para buscar tweets con un query

In [ ]:
class TwitterClient(object): 
    ''' 
    Generic Twitter Class for sentiment analysis. 
    '''
    def __init__(self): 
        ''' 
        Class constructor or initialization method. 
        '''
        # keys and tokens from the Twitter Dev Console 
        consumer_key = 'XXXXXXXXXXXXXXXXXXXXXXXX'
        consumer_secret = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXX'
        access_token = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXX'
        access_token_secret = 'XXXXXXXXXXXXXXXXXXXXXXXXX'
  
        # attempt authentication 
        try: 
            # create OAuthHandler object 
            self.auth = OAuthHandler(consumer_key, consumer_secret) 
            # set access token and secret 
            self.auth.set_access_token(access_token, access_token_secret) 
            # create tweepy API object to fetch tweets 
            self.api = tweepy.API(self.auth) 
        except: 
            print("Error: Authentication Failed") 
  
    def clean_tweet(self, tweet): 
        ''' 
        Utility function to clean tweet text by removing links, special characters 
        using simple regex statements. 
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) 
                                    |(\w+:\/\/\S+)", " ", tweet).split()) 
  
    def get_tweet_sentiment(self, tweet): 
        ''' 
        Utility function to classify sentiment of passed tweet 
        using textblob's sentiment method 
        '''
        # create TextBlob object of passed tweet text 
        analysis = TextBlob(self.clean_tweet(tweet)) 
        # set sentiment 
        if analysis.sentiment.polarity > 0: 
            return 'positive'
        elif analysis.sentiment.polarity == 0: 
            return 'neutral'
        else: 
            return 'negative'
  
    def get_tweets(self, query, count = 10): 
        ''' 
        Main function to fetch tweets and parse them. 
        '''
        # empty list to store parsed tweets 
        tweets = [] 
  
        try: 
            # call twitter api to fetch tweets 
            fetched_tweets = self.api.search(q = query, count = count) 
  
            # parsing tweets one by one 
            for tweet in fetched_tweets: 
                # empty dictionary to store required params of a tweet 
                parsed_tweet = {} 
  
                # saving text of tweet 
                parsed_tweet['text'] = tweet.text 
                # saving sentiment of tweet 
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text) 
  
                # appending parsed tweet to tweets list 
                if tweet.retweet_count > 0: 
                    # if tweet has retweets, ensure that it is appended only once 
                    if parsed_tweet not in tweets: 
                        tweets.append(parsed_tweet) 
                else: 
                    tweets.append(parsed_tweet) 
  
            # return parsed tweets 
            return tweets 
  
        except tweepy.TweepError as e: 
            # print error (if any) 
            print("Error : " + str(e)) 

In [ ]:
# creating object of TwitterClient Class 
api = TwitterClient() 
# calling function to get tweets 
tweets = api.get_tweets(query = 'Donald Trump', count = 200) 

# picking positive tweets from tweets 
ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive'] 
# percentage of positive tweets 
print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets))) 
# picking negative tweets from tweets 
ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative'] 
# percentage of negative tweets 
print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets))) 
# percentage of neutral tweets 
print("Neutral tweets percentage: {} % \ 
    ".format(100*len(tweets - ntweets - ptweets)/len(tweets))) 

# printing first 5 positive tweets 
print("\n\nPositive tweets:") 
for tweet in ptweets[:10]: 
    print(tweet['text']) 

# printing first 5 negative tweets 
print("\n\nNegative tweets:") 
for tweet in ntweets[:10]: 
    print(tweet['text']) 